In [ ]:
import xarray as xr
from glob import glob
from datetime import datetime
import yaml
import os

import plotting

try:
    from ecco_pipeline.conf.global_settings import GRIDS, OUTPUT_DIR
    from ecco_pipeline.utils.file_utils import get_date
    import ecco_pipeline.transformations.transformation as Transformation
except:
    os.chdir(os.path.realpath(os.path.join(os.path.abspath(''), '../../..')))
    from ecco_pipeline.conf.global_settings import GRIDS, OUTPUT_DIR
    from ecco_pipeline.utils.file_utils import get_date
    import ecco_pipeline.transformations.transformation as Transformation

In [ ]:
ds_name = ""
hemisphere = ""
with open(f'conf/ds_configs/{ds_name}.yaml', 'r') as stream:
    config = yaml.load(stream, yaml.Loader)

# Load sample granule

In [ ]:
ds_granules_dir = f'{OUTPUT_DIR}/{ds_name}/harvested_granules/'
ds_granules = glob(f'{ds_granules_dir}/**/*.nc*')
if hemisphere:
    ds_granules = [g for g in ds_granules if hemisphere in g]
sample_granule_path = ds_granules[0]
print(sample_granule_path)

In [ ]:
filename_date_str = get_date(config['filename_date_regex'], sample_granule_path.split('/')[-1])
granule_date = datetime.strptime(filename_date_str, config['filename_date_fmt']).strftime('%Y-%m-%dT%H:%M:%SZ')
ds = xr.open_dataset(sample_granule_path)
ds

# Load grid

In [ ]:
grids = glob(f'grids/*.nc*')
grids.sort()
f'Available grids: {grids}'

In [ ]:
grid_ds = xr.open_dataset(grids[1])
grid_ds

In [ ]:
T = Transformation.Transformation(config, sample_granule_path, granule_date)

# Make mapping factors

In [ ]:
factors = T.make_factors(grid_ds)

# Transformation

In [ ]:
field_ds_list = T.transform(grid_ds, factors, ds, config)

In [ ]:
field_ds = field_ds_list[0][0]
field_ds

# Visualization

In [ ]:
for field_ds in field_ds_list:
    plotting.make_plot(field_ds[0][list(field_ds[0].data_vars)[0]])